# Rocket launch analysis

<h1>Table of contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#API-data" data-toc-modified-id="API-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>API data</a></span></li><li><span><a href="#Webscraping-data" data-toc-modified-id="Webscraping-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Webscraping data</a></span></li><li><span><a href="#Data-union" data-toc-modified-id="Data-union-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data union</a></span></li></ul></div>

## Introduction



## API data

## Webscraping data

## Data union